Wszystko jest obiektem.

Duck Typing - liczy się interfejs a nie typ

W pythonie 2 wyróżna się 2 typy klass - old style i new style. Klasy starego typu są odradzane, jako że są dosyć ograniczone. Różnoca pomiędzy klasą nowego a starego typu to dziedziczenie po *object* (klasy nowego typu)


Definicja klasy:

In [ ]:
# klasa o nazwie Nazwa dziedzicząca po object i nie posiadająca zadnych atrybutóœ i metod zdefiniowanych przez użytkownika

class Nazwa(object):
    pass


Wszystkie atrybuty klasy i instancji są publiczne, a określanie atrybutów jako prywatne i protected to tylko konwencja nazewnictwa:

atrybuty zaczynające się od pojedyńczego podkreślnika \_atrybut - są to atrybuty protected
atrybuty zaczynające się od dwóch podkreslników \_\_atrybut - są to atrybuty prywatne - Python wspiera prywatnosć atrybutów przez mechanizm *name mangling*

atrybuty zaczynające się i kończące \_\_ (tzw. dunder) to specjalne atrybuty zarezerwowane dla interpretera (np. przeciążanie operatorów itp.)


In [ ]:
class A(object):
    def __init__(self, x):
        self.__x = x
        self.y = 2 * x
        self._z = x ** 2
        
a = A(1)
# name mangling przed nazwą atrybutu doczepia _ z nazwą klasy (żeby uniknąć kolizji w klasach dziedziczących)
dir(a)

Atrybuty obiektu przechowywane są w specjalnym słowniku \_\_dict\_\_. Słownik ten możńa modyfikować:

In [ ]:
print A.__dict__

print a.__dict__

a.__dict__['a'] = 13
print a.__dict__

## Monkey Patching
  
To dodawanei metod po definicji klasy, w praktyce wyglada to następująco:

In [ ]:
class A(object):
    pass

a = A()

def f(self):
    print self
    
A.f = f

a.f()

Jak dodajemy metody do instancji t obez parametru self:

In [ ]:
class A(object):
    pass

a = A()

def f():
    print 'f'
    
a.f = f

a.f()

## Magiczne (dunder) Metody

\_\_new\_\_ - tworzenie obiektu

\_\_init\_\_ - inicjalizacja

\_\_str\_\_, \_\_repr\_\_, \_\_unicode\_\_, \_\_format\_\_ - reprezentacja tekstowa

\_\_iter\_\_ - rzutowanie na listę/krotkę


### 3 typy operatorów:
* "normalne": \_\_add\_\_, \_\_mul\_\_
* "i" - inplace: \_\_iadd\_\_, \_\_imul\_\_
* "r" - reversed: \_\_radd\_\_, \_\_rmul\_\_


\_\_call\_\_ - wywoływanie obiektu jak funkcji

\_\_getitem\_\_, \_\_setitem\_\_, \_\_delitem\_\_ - dostęp jak do słownika

\_\_getattr\_\_, \_\_setattr\_\_, \_\_delattr\_\_, \_\_getattribute\_\_ - dostęp do atrybutów

\_\_hash\_\_ - wyznaczanie hashu obiektu - jeżeli 


### sekwencje
definiuje:

\_\_len\_\_ - długosć sekwencji
\_\_getitem\_\_ - wybieranie elementu z sekwencji - liczba lub slice

jeżeli sekwencja nie ma metody \_\_iter\_\_ to iterowanie następuje przez wrzucanie kolejnych liczb całkowitych od 0 do metody \_\_getitem\_\_ aż do skutku.

sekwencja może dodatkowo zdefiniować metodę \_\_conteins\_\_ do obsługi operatora **in**


### obiekt iterowalny
definiuje:

\_\_iter\_\_ - ma zwracać iterator a nie sekwencję
\_\_getitem\_\_


### iterator:
definiuje:

\_\_next\_\_, next - zwraca kolejny element obiektu po którym ma iterować
\_\_iter\_\_ - zwraca self



## NotImplemented vs NotImplementedError
NotImplemented jest obiektem, który powinna zwracać metoda jezeli nie implementuje danej operacji (np dodawanie), ponieważ wtedy interpreter jest wstanie podjąć akcje mające na celu zwrócenie wyniku.

NotImplementedError to bład, który jest wyrzucany, kiedy interpreter nie jest juz w stanie zapewnić wyniku operacji - można go obsłuzyć:

In [ ]:
class A(object):    
    def __eq__(self, other):
        print 'A.__eq__'
        if not isinstance(other, A):
            return NotImplemented
        return True
        

class B(object):
    def __eq__(self, other):
        print 'B.__eq__'
        if not isinstance(other, B):
            return False
        return True
    
a = A()
b = B()

a == b

## Atrybuty Klasy

Tak jak instancja ma atrybuty, tak i klasa może mieć atrybuty.

Atrybuty klasą są współdzielone pomiędzy instancjami.

Atrybuty klasy definiowane są w bloku klasy:

In [ ]:
class A(object):
    x = 2
    y = 3

Do atrybutów klasy można odwoływać się przez klasę i przez instancję (jezeli instancja nie ma juz zdefinowanych atrybutów o tej samej nazwie)

In [ ]:
a = A()

print A.x
print a.x

a.x = 4
print a.x

In [ ]:
b = A()
print b.x

print a.y
print b.y

A.y = 33
print a.y
print b.y

### classmethods i staticmethods

Poza standardowymi metodami, istnieja też metody statyczne i klasy.

classmethods przyjmują jako pierwszy argument klasę (a nie instancję) a metody statyczne nie przyjmują żadnej z powyższych. Metody statyczne i klasowe tworzy się używając dekoratorów.

In [ ]:
class A(object):
    @classmethod
    def f(cls):
        print cls
    
    @staticmethod
    def g():
        print 'g'
        

a = A()
a.f()
A.f()
a.g()

## Dziedziczenie

W Pythonie klasy bazowe są określane w nawiasie koło nazwy klasy. W Pythonie jedna klasa może mieć wiele klas bazowych.
W celu określenia kolejności poszukiwania metod używa się mro (method resolution order)

In [ ]:
class A(object):
    pass

class B(object):
    pass

class C(A):
        pass
        
class D(B):
        pass
        
class E(C, D):
    pass
        
        
print E.mro()

In [ ]:
# metoda super wywołuje inną metodę z kalsy bazowej

class A(object):
    def f(self):
        print 'A.f'
        super(A, self).f()

class B(object):
    def f(self):
        print 'B.f'
        super(B, self).f()

class C(A):
    def f(self):
        print 'C.f'
        super(C, self).f()
        
class D(B):
    def f(self):
        print 'D.f'
        super(D, self).f()
        
class E(C, D):
    def f(self):
        print 'E.f'
        super(E, self).f()
        
class F(C, D):
    def f(self):
        print 'F.f'
        super(C, self).f()
        
        
print E.mro()
try:
    E().f()
except:
    pass

print F.mro()
try:
    F().f()
except:
    pass

## Managery kontekstu

Managery kontekstu funkcjonują w połaczeniu z wyrażeniem *with*.
Interfejs managera to 2 metody: \_\_enter\_\_ i \_\_exit\_\_.

Obiekt, który będzie zwrócony w metodzie enter będzie przypisany do zmiennej po 'as'

In [ ]:
class Manager(object):
    def __enter__(self):
        print 'Manager.__enter__'
        return 22
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is KeyboardInterrupt:
            print 'ctrl c'
            return True
        print 'Exception not handled'
        return False
        

with Manager() as x:
    print x
    raise KeyboardInterrupt('stop')
    
    
with Manager() as x:
    print x
    {}[1]
    